In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.concat([
    pd.read_csv(f"./Driving_Behavior_Dataset/Features_By_Window_Size/sero_features_{i}.csv")
    for i in range(4, 21)
], ignore_index=True)



Index(['Target', 'AccMeanX', 'AccMeanY', 'AccMeanZ', 'AccCovX', 'AccCovY',
       'AccCovZ', 'AccSkewX', 'AccSkewY', 'AccSkewZ', 'AccKurtX', 'AccKurtY',
       'AccKurtZ', 'AccSumX', 'AccSumY', 'AccSumZ', 'AccMinX', 'AccMinY',
       'AccMinZ', 'AccMaxX', 'AccMaxY', 'AccMaxZ', 'AccVarX', 'AccVarY',
       'AccVarZ', 'AccMedianX', 'AccMedianY', 'AccMedianZ', 'AccStdX',
       'AccStdY', 'AccStdZ', 'GyroMeanX', 'GyroMeanY', 'GyroMeanZ', 'GyroCovX',
       'GyroCovY', 'GyroCovZ', 'GyroSkewX', 'GyroSkewY', 'GyroSkewZ',
       'GyroSumX', 'GyroSumY', 'GyroSumZ', 'GyroKurtX', 'GyroKurtY',
       'GyroKurtZ', 'GyroMinX', 'GyroMinY', 'GyroMinZ', 'GyroMaxX', 'GyroMaxY',
       'GyroMaxZ', 'GyroVarX', 'GyroVarY', 'GyroVarZ', 'GyroMedianX',
       'GyroMedianY', 'GyroMedianZ', 'GyroStdX', 'GyroStdY', 'GyroStdZ'],
      dtype='str')